# Getting the US Dollar exchange rate reports

The first step is to get the US Dollar exchange rate reporst that happens throwout the day. There are three types of reports
- Abertura(opening)
- Intermediary(Intermediário)
- Closing(Fechamento)

We are getting the dataset from the API: [Banco Central do Brasil Dollar API](https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/aplicacao#!/)



In [10]:
import requests
import numpy as np

from pandas.io.json import json_normalize

request = requests.get("https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='USD'&@dataInicial='07-01-2018'&@dataFinalCotacao='06-28-2019'&$format=json&$select=cotacaoCompra,dataHoraCotacao,tipoBoletim")

dolar_rates_json = request.json()

dolar_rates = json_normalize(dolar_rates_json['value'])

dolar_rates.tail(5)

,cotacaoCompra,dataHoraCotacao,tipoBoletim
1234,3.8297,2019-06-28 10:02:17.893,Abertura
1235,3.8347,2019-06-28 11:05:18.692,Intermediário
1236,3.8300,2019-06-28 12:06:28.871,Intermediário
1237,3.8319,2019-06-28 13:06:29.675,Intermediário
1238,3.8316,2019-06-28 13:06:29.684,Fechamento


# Translatting the dataframe to english

Simply translatting the portuguese words to english.

In [11]:
dolar_rates.columns = ['usd_quotation', 'quote_timestamp', 'report_type']
dolar_rates = dolar_rates.replace('Abertura', 'opening')
dolar_rates = dolar_rates.replace('Intermediário', 'intermediary')
dolar_rates = dolar_rates.replace('Fechamento', 'closing')

dolar_rates.tail(5)

,usd_quotation,quote_timestamp,report_type
1234,3.8297,2019-06-28 10:02:17.893,opening
1235,3.8347,2019-06-28 11:05:18.692,intermediary
1236,3.8300,2019-06-28 12:06:28.871,intermediary
1237,3.8319,2019-06-28 13:06:29.675,intermediary
1238,3.8316,2019-06-28 13:06:29.684,closing


# Categorizing US Dollar ratings

Now we are categorizing each rating, giving them one of three classes:

1. "UP" - If the USD_QUOTATION is greater than the previous value
2. "STEADY"- If the USD_QUOTATION is equal to the previous value
3. "DOWN" - If the USD_QUOTATION is smaller than the previous value

In [12]:
dolar_rates['variation'] = np.where(dolar_rates['usd_quotation'].diff() > 0, 'up', 
                           np.where(dolar_rates['usd_quotation'].diff() < 0, 'down', 'steady'))

dolar_rates = dolar_rates.drop(['usd_quotation', 'report_type'], axis=1)

dolar_rates.tail(5)

,quote_timestamp,variation
1234,2019-06-28 10:02:17.893,down
1235,2019-06-28 11:05:18.692,up
1236,2019-06-28 12:06:28.871,down
1237,2019-06-28 13:06:29.675,up
1238,2019-06-28 13:06:29.684,down


In [13]:
dolar_rates.to_csv('../datasets/pre-processed/dolar_rates.csv', index= False, encoding='utf-8')